# 移除重复数据

In [1]:
import numpy as np
import pandas as pd

In [2]:
data = pd.DataFrame({'k1': ['one', 'two'] * 3 + ['two'],
                   'k2': [1, 1, 2, 3, 3, 4, 4]})

In [3]:
data

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4
6,two,4


In [4]:
#DataFrame的duplicated方法返回一个布尔型Series，表示各行是否是重复行（前面出现过的行）：
data.duplicated()

0    False
1    False
2    False
3    False
4    False
5    False
6     True
dtype: bool

In [5]:
#还有一个与此相关的drop_duplicates方法，它会返回一个DataFrame，重复的数组会标为False：
data.drop_duplicates()

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4


In [6]:
#这两个方法默认会判断全部列，你也可以指定部分列进行重复项判断。假设我们还有一列值，且只希望根据k1列过滤重复项：
data['v1'] = range(7)

In [7]:
data.drop_duplicates(['k1'])

,k1,k2,v1
0,one,1,0
1,two,1,1


In [8]:
#duplicated和drop_duplicates默认保留的是第一个出现的值组合。传入keep='last'则保留最后一个：
data.drop_duplicates(['k1', 'k2'], keep='last')

,k1,k2,v1
0,one,1,0
1,two,1,1
2,one,2,2
3,two,3,3
4,one,3,4
6,two,4,6


# 利用函数或映射进行数据转换

In [9]:
#对于许多数据集，你可能希望根据数组、Series或DataFrame列中的值来实现转换工作。我们来看看下面这组有关肉类的数据：

In [10]:
data = pd.DataFrame({'food': ['bacon', 'pulled pork', 'bacon',
           'Pastrami', 'corned beef', 'Bacon',
                  'pastrami', 'honey ham', 'nova lox'],
            'ounces': [4, 3, 12, 6, 7.5, 8, 3, 5, 6]})

In [11]:
data

,food,ounces
0,bacon,4.0
1,pulled pork,3.0
2,bacon,12.0
3,Pastrami,6.0
4,corned beef,7.5
5,Bacon,8.0
6,pastrami,3.0
7,honey ham,5.0
8,nova lox,6.0


In [12]:
#假设你想要添加一列表示该肉类食物来源的动物类型。我们先编写一个不同肉类到动物的映射：
meat_to_animal = {
  'bacon': 'pig',
  'pulled pork': 'pig',
  'pastrami': 'cow',
  'corned beef': 'cow',
  'honey ham': 'pig',
  'nova lox': 'salmon'
}

In [13]:
#Series的map方法可以接受一个函数或含有映射关系的字典型对象，但是这里有一个小问题，即有些肉类的首字母大写了，而另一些则没有。因此，我们还需要使用Series的str.lower方法，将各个值转换为小写：
lowercased = data['food'].str.lower()

In [14]:
lowercased

0          bacon
1    pulled pork
2          bacon
3       pastrami
4    corned beef
5          bacon
6       pastrami
7      honey ham
8       nova lox
Name: food, dtype: object

In [15]:
data['animal'] = lowercased.map(meat_to_animal)

In [16]:
data

,food,ounces,animal
0,bacon,4.0,pig
1,pulled pork,3.0,pig
2,bacon,12.0,pig
3,Pastrami,6.0,cow
4,corned beef,7.5,cow
5,Bacon,8.0,pig
6,pastrami,3.0,cow
7,honey ham,5.0,pig
8,nova lox,6.0,salmon


In [17]:
#我们也可以传入一个能够完成全部这些工作的函数：
data['food'].map(lambda x: meat_to_animal[x.lower()])
#lower()函数描述：将字符串中的所有大写字母转换为小写字母。

0       pig
1       pig
2       pig
3       cow
4       cow
5       pig
6       cow
7       pig
8    salmon
Name: food, dtype: object

In [18]:
#使用map是一种实现元素级转换以及其他数据清理工作的便捷方式。

# 替换值

In [19]:
#利用fillna方法填充缺失数据可以看做值替换的一种特殊情况。前面已经看到，map可用于修改对象的数据子集，而replace则提供了一种实现该功能的更简单、更灵活的方式。我们来看看下面这个Series：

In [20]:
data = pd.Series([1., -999., 2., -999., -1000., 3.])
data

0       1.0
1    -999.0
2       2.0
3    -999.0
4   -1000.0
5       3.0
dtype: float64

In [21]:
#-999这个值可能是一个表示缺失数据的标记值。要将其替换为pandas能够理解的NA值，我们可以利用replace来产生一个新的Series（除非传入inplace=True）：
data.replace(-999,np.nan)

0       1.0
1       NaN
2       2.0
3       NaN
4   -1000.0
5       3.0
dtype: float64

In [22]:
#如果你希望一次性替换多个值，可以传入一个由待替换值组成的列表以及一个替换值：
data.replace([-999,-1000],np.nan)

0    1.0
1    NaN
2    2.0
3    NaN
4    NaN
5    3.0
dtype: float64

In [23]:
#要让每个值有不同的替换值，可以传递一个替换列表：
data.replace([-999, -1000], [np.nan, 0])

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

In [24]:
#传入的参数也可以是字典：
data.replace({-999: np.nan, -1000: 0})

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

In [25]:
###data.replace方法与data.str.replace不同，后者做的是字符串的元素级替换

# 重命名轴索引

In [26]:
#跟Series中的值一样，轴标签也可以通过函数或映射进行转换，从而得到一个新的不同标签的对象。轴还可以被就地修改，而无需新建一个数据结构。接下来看看下面这个简单的例子：

In [27]:
data = pd.DataFrame(np.arange(12.).reshape((3,4)),
                   index=['Ohio', 'Colorado', 'New York'],
                   columns=['one', 'two', 'three', 'four'])

In [28]:
data

,one,two,three,four
Ohio,0.0,1.0,2.0,3.0
Colorado,4.0,5.0,6.0,7.0
New York,8.0,9.0,10.0,11.0


In [29]:
#跟Series一样，轴索引也有一个map方法：
transform = lambda x: x[:4].upper()#将行的索引名字变为大写且只取前四个字母

In [30]:
data.index.map(transform)

Index(['OHIO', 'COLO', 'NEW '], dtype='object')

In [31]:
#你可以将其赋值给index，这样就可以对DataFrame进行就地修改：
data.index = data.index.map(transform)

In [32]:
data

,one,two,three,four
OHIO,0.0,1.0,2.0,3.0
COLO,4.0,5.0,6.0,7.0
NEW,8.0,9.0,10.0,11.0


In [33]:
#如果想要创建数据集的转换版（而不是修改原始数据），比较实用的方法是rename：
data.rename(index=str.title, columns=str.upper)

,ONE,TWO,THREE,FOUR
Ohio,0.0,1.0,2.0,3.0
Colo,4.0,5.0,6.0,7.0
New,8.0,9.0,10.0,11.0


In [34]:
#特别说明一下，rename可以结合字典型对象实现对部分轴标签的更新：
data.rename(index={'OHIO': 'INDIANA'},
          columns={'three': 'peekaboo'})

,one,two,peekaboo,four
INDIANA,0.0,1.0,2.0,3.0
COLO,4.0,5.0,6.0,7.0
NEW,8.0,9.0,10.0,11.0


In [35]:
#rename可以实现复制DataFrame并对其索引和列标签进行赋值。如果希望就地修改某个数据集，传入inplace=True即可：
data.rename(index={'OHIO': 'INDIANA'}, inplace=True)

In [36]:
data

,one,two,three,four
INDIANA,0.0,1.0,2.0,3.0
COLO,4.0,5.0,6.0,7.0
NEW,8.0,9.0,10.0,11.0


# 离散化和面元划分

In [37]:
#为了便于分析，连续数据常常被离散化或拆分为“面元”（bin）。假设有一组人员数据，而你希望将它们划分为不同的年龄组：

In [38]:
ages = [20, 22, 25, 27, 21, 23, 37, 31, 61, 45, 41, 32]

In [39]:
#接下来将这些数据划分为“18到25”、“26到35”、“35到60”以及“60以上”几个面元。要实现该功能，你需要使用pandas的cut函数：

In [40]:
bins = [18, 25, 35, 60, 100]

In [41]:
cats = pd.cut(ages, bins)

In [42]:
cats

[(18, 25], (18, 25], (18, 25], (25, 35], (18, 25], ..., (25, 35], (60, 100], (35, 60], (35, 60], (25, 35]]
Length: 12
Categories (4, interval[int64]): [(18, 25] < (25, 35] < (35, 60] < (60, 100]]

In [43]:
#pandas返回的是一个特殊的Categorical对象。结果展示了pandas.cut划分的面元。你可以将其看做一组表示面元名称的字符串。它的底层含有一个表示不同分类名称的类型数组，以及一个codes属性中的年龄数据的标签：
cats.codes

array([0, 0, 0, 1, 0, 0, 2, 1, 3, 2, 2, 1], dtype=int8)

In [44]:
cats.categories

IntervalIndex([(18, 25], (25, 35], (35, 60], (60, 100]],
              closed='right',
              dtype='interval[int64]')

In [45]:
pd.value_counts(cats)
#pd.value_counts(cats)是pandas.cut结果的面元计数。

(18, 25]     5
(35, 60]     3
(25, 35]     3
(60, 100]    1
dtype: int64

In [46]:
#跟“区间”的数学符号一样，圆括号表示开端，而方括号则表示闭端（包括）。哪边是闭端可以通过right=False进行修改：
pd.cut(ages, [18, 26, 36, 61, 100], right=False)

[[18, 26), [18, 26), [18, 26), [26, 36), [18, 26), ..., [26, 36), [61, 100), [36, 61), [36, 61), [26, 36)]
Length: 12
Categories (4, interval[int64]): [[18, 26) < [26, 36) < [36, 61) < [61, 100)]

In [47]:
#你可 以通过传递一个列表或数组到labels，设置自己的面元名称：
group_names = ['Youth', 'YoungAdult', 'MiddleAged', 'Senior']

In [48]:
pd.cut(ages, bins, labels=group_names)

[Youth, Youth, Youth, YoungAdult, Youth, ..., YoungAdult, Senior, MiddleAged, MiddleAged, YoungAdult]
Length: 12
Categories (4, object): [Youth < YoungAdult < MiddleAged < Senior]

In [49]:
#如果向cut传入的是面元的数量而不是确切的面元边界，则它会根据数据的最小值和最大值计算等长面元。下面这个例子中，我们将一些均匀分布的数据分成四组：

In [50]:
data = np.random.rand(20)

In [51]:
pd.cut(data, 4, precision=2)#选项precision=2，限定小数只有两位。

[(0.74, 0.98], (0.74, 0.98], (0.26, 0.5], (0.26, 0.5], (0.26, 0.5], ..., (0.5, 0.74], (0.74, 0.98], (0.5, 0.74], (0.023, 0.26], (0.023, 0.26]]
Length: 20
Categories (4, interval[float64]): [(0.023, 0.26] < (0.26, 0.5] < (0.5, 0.74] < (0.74, 0.98]]

In [52]:
#qcut是一个非常类似于cut的函数，它可以根据样本分位数对数据进行面元划分。根据数据的分布情况，cut可能无法使各个面元中含有相同数量的数据点。而qcut由于使用的是样本分位数，因此可以得到大小基本相等的面元：

In [53]:
data = np.random.randn(1000) 

In [54]:
cats = pd.qcut(data, 4)

In [55]:
cats

[(-0.0223, 0.757], (-3.163, -0.672], (-0.0223, 0.757], (-3.163, -0.672], (-3.163, -0.672], ..., (-3.163, -0.672], (0.757, 3.284], (-0.0223, 0.757], (-3.163, -0.672], (-0.672, -0.0223]]
Length: 1000
Categories (4, interval[float64]): [(-3.163, -0.672] < (-0.672, -0.0223] < (-0.0223, 0.757] < (0.757, 3.284]]

In [56]:
pd.value_counts(cats)#着重体现点

(0.757, 3.284]       250
(-0.0223, 0.757]     250
(-0.672, -0.0223]    250
(-3.163, -0.672]     250
dtype: int64

In [57]:
#与cut类似，你也可以传递自定义的分位数（0到1之间的数值，包含端点）：
pd.qcut(data, [0, 0.1, 0.5, 0.9, 1.])

[(-0.0223, 1.362], (-1.312, -0.0223], (-0.0223, 1.362], (-1.312, -0.0223], (-1.312, -0.0223], ..., (-3.163, -1.312], (-0.0223, 1.362], (-0.0223, 1.362], (-3.163, -1.312], (-1.312, -0.0223]]
Length: 1000
Categories (4, interval[float64]): [(-3.163, -1.312] < (-1.312, -0.0223] < (-0.0223, 1.362] < (1.362, 3.284]]

# 检测和过滤异常值

In [58]:
#过滤或变换异常值（outlier）在很大程度上就是运用数组运算。来看一个含有正态分布数据的DataFrame：

In [59]:
data = pd.DataFrame(np.random.randn(1000,4))

In [60]:
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,-0.025510,-0.004472,-0.024678,0.011409
std,1.022527,1.002767,0.963691,1.049986
min,-2.990484,-2.825290,-2.853343,-3.609369
25%,-0.681498,-0.648106,-0.637811,-0.682569
50%,-0.057867,-0.029252,-0.034367,0.034830
75%,0.624769,0.650512,0.628190,0.714447
max,2.879711,3.710132,2.835332,3.776327


In [61]:
#假设你想要找出某列中绝对值大小超过3的值：
col = data[2]

In [62]:
col[np.abs(col)>3]

Series([], Name: 2, dtype: float64)

In [63]:
#要选出全部含有“超过3或－3的值”的行，你可以在布尔型DataFrame中使用any方法：
data[(np.abs(data) > 3).any(1)]#any(1)就是对每一个列进行操作

,0,1,2,3
59,0.583543,3.710132,-0.568536,-0.024283
78,0.255820,-1.003164,-1.992315,-3.609369
204,-1.283677,3.278182,1.887252,-0.085269
220,2.525681,3.178281,0.027636,0.850105
361,1.413204,-1.378847,-0.265483,-3.180637
392,0.959894,0.006623,1.095989,-3.360915
394,1.827259,-1.387827,1.440461,-3.197683
510,1.472938,-0.816123,2.312921,3.776327
709,0.016224,0.349865,0.150427,3.006276
763,-0.265225,3.104951,-0.806477,1.153819


In [64]:
#根据这些条件，就可以对值进行设置。下面的代码可以将值限制在区间－3到3以内：
data[np.abs(data) > 3] = np.sign(data) * 3#np.sign()符号函数，若初始值为负数则置为-1，为则正数置为1

In [65]:
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,-0.025510,-0.005827,-0.024678,0.012468
std,1.022527,0.998380,0.963691,1.041758
min,-2.990484,-2.825290,-2.853343,-3.000000
25%,-0.681498,-0.648106,-0.637811,-0.682569
50%,-0.057867,-0.029252,-0.034367,0.034830
75%,0.624769,0.650512,0.628190,0.714447
max,2.879711,3.000000,2.835332,3.000000


In [66]:
#根据数据的值是正还是负，np.sign(data)可以生成1和-1：
np.sign(data).head()

,0,1,2,3
0,1.0,1.0,-1.0,-1.0
1,-1.0,-1.0,-1.0,-1.0
2,1.0,1.0,-1.0,1.0
3,-1.0,1.0,-1.0,1.0
4,-1.0,-1.0,1.0,-1.0


# 排列和随机采样

In [67]:
#利用numpy.random.permutation函数可以轻松实现对Series或DataFrame的列的排列工作（permuting，随机重排序）。通过需要排列的轴的长度调用permutation，可产生一个表示新顺序的整数数组：

In [68]:
df = pd.DataFrame(np.arange(5*4).reshape((5,4)))

In [69]:
sampler = np.random.permutation(5)

In [70]:
sampler

array([0, 2, 4, 3, 1])

In [71]:
#然后就可以在基于iloc的索引操作或take函数中使用该数组了：

In [72]:
df

,0,1,2,3
0,0,1,2,3
1,4,5,6,7
2,8,9,10,11
3,12,13,14,15
4,16,17,18,19


In [73]:
df.take(sampler)

,0,1,2,3
0,0,1,2,3
2,8,9,10,11
4,16,17,18,19
3,12,13,14,15
1,4,5,6,7


In [74]:
#如果不想用替换的方式选取随机子集，可以在Series和DataFrame上使用sample方法：

In [75]:
df.sample(n=3)

,0,1,2,3
0,0,1,2,3
4,16,17,18,19
1,4,5,6,7


In [76]:
#要通过替换的方式产生样本（允许重复选择），可以传递replace=True到sample：

In [77]:
choices = pd.Series([5, 7, -1, 6, 4])

In [78]:
draws = choices.sample(n=10, replace=True)#replace=True表示可以重复

In [79]:
draws

4    4
2   -1
0    5
2   -1
1    7
4    4
3    6
0    5
3    6
2   -1
dtype: int64

# 计算指标/哑变量

In [80]:
#另一种常用于统计建模或机器学习的转换方式是：将分类变量（categorical variable）转换为“哑变量”或“指标矩阵”。

In [81]:
#如果DataFrame的某一列中含有k个不同的值，则可以派生出一个k列矩阵或DataFrame（其值全为1和0）。pandas有一个get_dummies函数可以实现该功能（其实自己动手做一个也不难）。使用之前的一个DataFrame例子：

In [82]:
df = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'b'],
      'data1': range(6)})

In [84]:
df

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,b,5


In [85]:
pd.get_dummies(df['key'])

,a,b,c
0,0,1,0
1,0,1,0
2,1,0,0
3,0,0,1
4,1,0,0
5,0,1,0


In [86]:
#有时候，你可能想给指标DataFrame的列加上一个前缀，以便能够跟其他数据进行合并。get_dummies的prefix参数可以实现该功能：
dummies = pd.get_dummies(df['key'], prefix='key')

In [87]:
df_with_dummy = df[['data1']].join(dummies)

In [88]:
df_with_dummy

,data1,key_a,key_b,key_c
0,0,0,1,0
1,1,0,1,0
2,2,1,0,0
3,3,0,0,1
4,4,1,0,0
5,5,0,1,0


In [89]:
#如果DataFrame中的某行同属于多个分类，则事情就会有点复杂。

In [90]:
mnames = ['movie_id', 'title', 'genres']

In [93]:
movies = pd.read_table('datasets\movielens\movies.dat', sep='::',header=None, names=mnames)
#seq标识分隔符，分隔符为\t ，即制表符，表示列与列之间用\t分开
#header=None 表示txt文件的第一行不是列的名字，是数据。若不写head=None，则读入txt数据时，会没了第一行

<ipython-input-93-a2d403c7a03e>:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  movies = pd.read_table('datasets\movielens\movies.dat', sep='::',header=None, names=mnames)


In [94]:
movies[:10]

,movie_id,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children's
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller


In [96]:
movies.shape

(3883, 3)

In [97]:
#要为每个genre添加指标变量就需要做一些数据规整操作。首先，我们从数据集中抽取出不同的genre值：

In [98]:
all_genres = []

In [99]:
for x in movies.genres:
    all_genres.extend(x.split('|'))

In [100]:
genres = pd.unique(all_genres)
#pd.unique()返回一个“独一无二”的值

In [101]:
genres

array(['Animation', "Children's", 'Comedy', 'Adventure', 'Fantasy',
       'Romance', 'Drama', 'Action', 'Crime', 'Thriller', 'Horror',
       'Sci-Fi', 'Documentary', 'War', 'Musical', 'Mystery', 'Film-Noir',
       'Western'], dtype=object)

In [102]:
#构建指标DataFrame的方法之一是从一个全零DataFrame开始：

In [105]:
zero_matrix = np.zeros((len(movies),len(genres)))

In [106]:
dummies = pd.DataFrame(zero_matrix,columns=genres)

In [107]:
#现在，迭代每一部电影，并将dummies各行的条目设为1。要这么做，我们使用dummies.columns来计算每个类型的列索引：

In [108]:
gen = movies.genres[0]

In [109]:
gen.split('|')

['Animation', "Children's", 'Comedy']

In [110]:
dummies.columns.get_indexer(gen.split('|'))

array([0, 1, 2], dtype=int64)

In [111]:
#然后，根据索引，使用.iloc设定值：

In [112]:
for i, gen in enumerate(movies.genres):#enumerate() 函数同时得到索引位置和对应值：
    indices = dummies.columns.get_indexer(gen.split('|'))
    dummies.iloc[i, indices] = 1

In [113]:
#然后，和以前一样，再将其与movies合并起来：

In [114]:
movies_windic = movies.join(dummies.add_prefix('Genre_'))

In [115]:
movies_windic.iloc[0]

movie_id                                       1
title                           Toy Story (1995)
genres               Animation|Children's|Comedy
Genre_Animation                                1
Genre_Children's                               1
Genre_Comedy                                   1
Genre_Adventure                                0
Genre_Fantasy                                  0
Genre_Romance                                  0
Genre_Drama                                    0
Genre_Action                                   0
Genre_Crime                                    0
Genre_Thriller                                 0
Genre_Horror                                   0
Genre_Sci-Fi                                   0
Genre_Documentary                              0
Genre_War                                      0
Genre_Musical                                  0
Genre_Mystery                                  0
Genre_Film-Noir                                0
Genre_Western       

In [116]:
#一个对统计应用有用的秘诀是：结合get_dummies和诸如cut之类的离散化函数：

In [118]:
np.random.seed(12345)
#np.random.seed()函数用于生成指定随机数。
#seed()被设置了之后，np,random.random()可以按顺序产生一组固定的数组，如果使用相同的seed()值，则每次生成的随机数都相同，如果不设置这个值，那么每次生成的随机数不同。但是，只在调用的时候seed()一下并不能使生成的随机数相同，需要每次调用都seed()一下，表示种子相同，从而生成的随机数相同。

In [119]:
values = np.random.rand(10)
###random.rand()函数根据给定维度生成[0,1)之间的数据，包含0，不包含1
###random.randn()返回一组服从正态分布的随机值。

In [120]:
values

array([0.92961609, 0.31637555, 0.18391881, 0.20456028, 0.56772503,
       0.5955447 , 0.96451452, 0.6531771 , 0.74890664, 0.65356987])

In [121]:
bins = [0, 0.2, 0.4, 0.6, 0.8, 1]

In [122]:
pd.get_dummies(pd.cut(values, bins))

,"(0.0, 0.2]","(0.2, 0.4]","(0.4, 0.6]","(0.6, 0.8]","(0.8, 1.0]"
0,0,0,0,0,1
1,0,1,0,0,0
2,1,0,0,0,0
3,0,1,0,0,0
4,0,0,1,0,0
5,0,0,1,0,0
6,0,0,0,0,1
7,0,0,0,1,0
8,0,0,0,1,0
9,0,0,0,1,0


In [123]:
#我们用numpy.random.seed，使这个例子具有确定性。